In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor

In [2]:
market = Market()
market.connect()
dates = market.retrieve_date_range()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
stuff = pd.to_datetime(dates["date"])

In [4]:
start = (stuff.max() + timedelta(days=1)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2021-12-10 2021-12-11


In [5]:
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        data = TiingoExtractor.extract(ticker,start,end)
        data["ticker"] = ticker
        market.store("prices",data)
    except Exception as e:
        print(str(e))
market.disconnect()

 16%|██████████████████▋                                                                                                | 82/505 [00:14<01:03,  6.70it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


 61%|█████████████████████████████████████████████████████████████████████                                             | 306/505 [00:54<00:41,  4.77it/s]

project_market prices documents must be a non-empty list


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 503/505 [01:27<00:00,  6.95it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment
Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [01:27<00:00,  5.77it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment
Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


In [6]:
data

In [7]:
analysis = []
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("prices",ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        periods = ["year","quarter","month","week"]
        ep = test.tail(1)["adjClose"].item()
        # periods = ["year"]
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(str(e))
market.disconnect()

 13%|███████████████                                                                                                    | 66/505 [00:04<00:30, 14.26it/s]

'date'


 16%|██████████████████▏                                                                                                | 80/505 [00:05<00:25, 16.85it/s]

'date'


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [00:35<00:00, 14.42it/s]

'date'
'date'
'date'
'date'


In [8]:
a = pd.DataFrame(analysis).merge(sp5.rename(columns={"Symbol":"ticker"})[["ticker","Security","GICS Sector"]],on="ticker",how="left")
a.to_csv("analysis.csv")

In [9]:
period = "day"
a.sort_values(f"{period}_gain",ascending=False).head(50)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector
353,0.628922,0.143637,0.143510,0.153924,0.156134,ORCL,Oracle,Information Technology
195,1.517606,0.514831,0.095506,0.116025,0.096065,F,Ford,Consumer Discretionary
75,0.519466,0.296153,0.138655,0.118057,0.082719,AVGO,Broadcom,Information Technology
127,0.478747,0.246448,0.054696,0.048050,0.065779,COST,Costco,Consumer Staples
210,0.560355,0.189723,0.088701,0.056317,0.060215,GM,General Motors,Consumer Discretionary
196,1.285174,0.116208,0.089661,0.138988,0.051229,FTNT,Fortinet,Information Technology
93,0.263488,0.244251,0.117840,0.093075,0.047816,CNC,Centene Corporation,Health Care
96,0.617121,0.000491,0.067726,0.038187,0.045284,CF,CF Industries,Materials
77,0.152253,0.212507,0.115372,0.069459,0.040445,CHRW,C. H. Robinson,Industrials
264,0.538745,0.214286,0.103896,0.081081,0.038485,JNPR,Juniper Networks,Information Technology


In [10]:
a.groupby("GICS Sector").mean().sort_values("day_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Consumer Staples,0.107171,0.052430,0.063550,0.016394,0.014641
Information Technology,0.266013,0.072064,0.043807,0.029132,0.011838
Energy,0.627820,0.048225,0.065682,0.026581,0.011214
Materials,0.284278,0.074610,0.056223,0.018974,0.008075
Consumer Discretionary,0.302234,0.069672,0.066360,0.019275,0.005711
Utilities,0.141742,0.066151,0.053936,0.009786,0.005591
Industrials,0.277595,0.061889,0.059076,0.013699,0.003674
Real Estate,0.468636,0.078765,0.056484,0.011610,0.002304
Health Care,0.182832,0.004899,0.045052,0.025199,0.001902
